# Credits

This is heavily influenced or copied from https://github.com/pytorch/tutorials

# Autograd: automatic differentiation

Central to all neural networks in PyTorch is the ``autograd`` package.
Let’s first briefly visit this, and we will then go to training our first neural network.

The `autograd` package provides automatic differentiation for all operations on Tensors.
It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.

Let us see this in more simple terms with some examples.

## 1. Tensor

`torch.Tensor` is the central class of the package. Setting the attribute `.requires_grad` to `True` will make the tensor "record" all operations on it. When you finish your computation you can call `.backward()` and have all the gradients computed automatically. The gradient for this tensor will be accumulated into the `.grad` attribute.

![autograd.Variable](https://github.com/Laegas/02456-deep-learning-with-PyTorch/blob/master/static_files/autograd-variable.png?raw=1)

There’s one more class which is very important for autograd implementation - a `Function`.

`Tensor` and `Function` are interconnected and build up an acyclic graph, that encodes a complete history of computation. Each tensor has a `.grad_fn` attribute that references a `Function` that has created the `Tensor` (except for Tensors created by the user - their `grad_fn` is `None`).

If you want to compute the derivatives, you can call `.backward()` on a Tensor. If `Tensor` is a scalar (i.e. it holds a one element data), you don’t need to specify any arguments to backward(), however if it has more elements, you need to specify a `gradient` argument that is a tensor of matching shape.

In [1]:
import torch

Create a tensor

In [2]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


Do a tensor operation:

In [3]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


`y` was created as a result of an operation, so it has a `grad_fn`.

In [4]:
print(y.grad_fn)

Do more operations on y

In [5]:
z = y * y * 3
out = z.mean()

print(z)
print(out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)


# Assignments

1. Create a Tensor that `requires_grad` of size (5, 5)
2. Sum the values in the Tensor

In [6]:
# 1.
y = torch.rand(5, 5, requires_grad=True)
print(y)
# 2.
sum(y)

tensor([[0.0110, 0.3028, 0.1713, 0.5439, 0.3475],
        [0.9201, 0.3719, 0.2809, 0.5559, 0.6020],
        [0.6975, 0.9158, 0.7730, 0.9770, 0.8252],
        [0.5203, 0.8083, 0.0952, 0.4425, 0.4135],
        [0.0725, 0.3105, 0.5449, 0.5839, 0.9817]], requires_grad=True)


tensor([2.2214, 2.7093, 1.8654, 3.1032, 3.1699], grad_fn=<AddBackward0>)

## 2. Gradients

Let’s backprop now. Because `out` contains a single scalar, `out.backward()` is equivalent to `out.backward(torch.tensor([1.0]))`

In [7]:
out.backward()

Print gradients d(out)/dx

In [8]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


You should have a matrix of `4.5`. Let’s denote the tensor `out` with $o$.

We have:
$o = \frac{1}{4}\sum_i z_i$, $z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$.

Therefore, $\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$,
hence $\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.

You can do many crazy things with autograd!

In [9]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([-220.4487, 1028.7584, -510.6211], grad_fn=<MulBackward0>)


In [10]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


**Read later** \
*Documentation* \
`Tensor`: https://pytorch.org/docs/stable/tensors.html \
`Function`: http://pytorch.org/docs/autograd

# Assignments

1. Define a tensor and set `requires_grad` to `True`
3. Multiply the tensor by 2 and assign the result to a new python variable (i.e. `x = result`)
4. Sum the variable's elements and assign to a new python variable
5. Print the gradients of all the variables
6. Now perform a backward pass on the last variable (NOTE: for each new python variable that you define, call `.retain_grad()`)
7. Print all gradients again
  - what did you notice?

In [19]:
# 1
print("#1")
tensor = torch.rand(4,5,requires_grad=True)
tensor.retain_grad()
print(tensor)

# 2
print("#2")
x = tensor * 2
x.retain_grad()
print(x)

# 3
print("#3")
sum = torch.sum(x)
sum.retain_grad()
print(sum)

# 4
print("#4")
print(tensor.grad, x.grad, sum.grad)

# 5
print("#5")
sum.backward()

# 6
print("#6")
print(tensor.grad, "\n", x.grad, "\n", sum.grad)

#1
tensor([[0.8766, 0.8557, 0.3684, 0.0307, 0.7579],
        [0.0831, 0.2670, 0.6875, 0.3331, 0.2307],
        [0.9817, 0.9588, 0.0309, 0.8982, 0.0013],
        [0.3113, 0.4055, 0.3470, 0.5425, 0.1111]], requires_grad=True)
#2
tensor([[1.7532, 1.7114, 0.7368, 0.0613, 1.5158],
        [0.1661, 0.5339, 1.3750, 0.6662, 0.4614],
        [1.9634, 1.9176, 0.0619, 1.7964, 0.0025],
        [0.6225, 0.8109, 0.6939, 1.0850, 0.2222]], grad_fn=<MulBackward0>)
#3
tensor(18.1576, grad_fn=<SumBackward0>)
#4
None None None
#5
#6
tensor([[2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.]]) 
 tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]) 
 tensor(1.)
